This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU is free of charge). Therefore, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In case you are facing issues, please read the following two documents first:

https://github.com/IBM/skillsnetwork/wiki/Environment-Setup

https://github.com/IBM/skillsnetwork/wiki/FAQ

Then, please feel free to ask:

https://coursera.org/learn/machine-learning-big-data-apache-spark/discussions/all

Please make sure to follow the guidelines before asking a question:

https://github.com/IBM/skillsnetwork/wiki/FAQ#im-feeling-lost-and-confused-please-help-me


If running outside Watson Studio, this should work as well. In case you are running in an Apache Spark context outside Watson Studio, please remove the Apache Spark setup in the first notebook cells.

In [ ]:
!pip install pyspark==2.4.5
!pip install --upgrade pip

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if not ('sc' in locals() or 'sc' in globals()):
    print('It seems you are note running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production')
    
    from pip import main
    main(['install', 'pyspark==2.4.5'])
    
    try:
        from pyspark import SparkContext, SparkConf
        from pyspark.sql import SparkSession
    except ImportError as e:
        print('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

    sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    
    spark = SparkSession \
        .builder \
        .getOrCreate()

It seems you are note running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Welcome to exercise two of “Apache Spark for Scalable Machine Learning on BigData”. In this exercise you’ll apply the basics of functional and parallel programming. 

Again, please use the following two links for your reference:
https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD
https://spark.apache.org/docs/latest/rdd-programming-guide.html

Let’s actually create a python function which decides whether a value is greater than 50 (True) or not (False).

In [2]:
rdd = sc.parallelize(range(100))

In [3]:
# please replace $$ with the correct characters
rdd.count()

100

In [4]:
rdd.sum()

4950

In [5]:
def gt50(i):
    if i > 50:
        return True
    else:
        return False

In [6]:
print(gt50(4))
print(gt50(51))

False
True


Let’s simplify this function

In [7]:
def gt50(i):
    return i > 50

In [8]:
print(gt50(4))
print(gt50(51))

False
True


Now let’s use the lambda notation to define the function.

In [9]:
gt50 = lambda i: i > 50

In [10]:
print(gt50(4))
print(gt50(51))

False
True


In [11]:
#let's shuffle our list to make it a bit more interesting
from random import shuffle
l = list(range(100))
shuffle(l)
rdd = sc.parallelize(l)

Let’s filter values from our list which are equals or less than 50 by applying our “gt50” function to the list using the “filter” function. Note that by calling the “collect” function, all elements are returned to the Apache Spark Driver. This is not a good idea for BigData, please use “.sample(10,0.1).collect()” or “take(n)” instead.

In [12]:
rdd.filter(gt50).collect()

[62,
 57,
 66,
 92,
 93,
 72,
 58,
 51,
 79,
 70,
 82,
 80,
 95,
 71,
 59,
 60,
 73,
 94,
 56,
 83,
 85,
 74,
 69,
 77,
 61,
 89,
 84,
 86,
 76,
 63,
 96,
 54,
 52,
 98,
 75,
 91,
 65,
 53,
 64,
 81,
 88,
 90,
 78,
 55,
 99,
 97,
 67,
 87,
 68]

We can also use the lambda function directly.

In [13]:
rdd.filter(lambda i: i > 50).collect()

[62,
 57,
 66,
 92,
 93,
 72,
 58,
 51,
 79,
 70,
 82,
 80,
 95,
 71,
 59,
 60,
 73,
 94,
 56,
 83,
 85,
 74,
 69,
 77,
 61,
 89,
 84,
 86,
 76,
 63,
 96,
 54,
 52,
 98,
 75,
 91,
 65,
 53,
 64,
 81,
 88,
 90,
 78,
 55,
 99,
 97,
 67,
 87,
 68]

Let’s consider the same list of integers. Now we want to compute the sum for elements in that list which are greater than 50 but less than 75. Please implement the missing parts. 

In [14]:
rdd.filter(lambda x: x > 50).filter(lambda x: x < 75).sum()

1500

You should see "1500" as answer. Now we want to know the sum of all elements. Please again, have a look at the API documentation and complete the code below in order to get the sum.

In [15]:
from pyspark.sql import Row

df = spark.createDataFrame([Row(id=1, value='value1'),Row(id=2, value='value2')])

# let's have a look what's inside
df.show()

# let's print the schema
df.printSchema()

+---+------+
| id| value|
+---+------+
|  1|value1|
|  2|value2|
+---+------+

root
 |-- id: long (nullable = true)
 |-- value: string (nullable = true)



In [16]:
# register dataframe as query table
df.createOrReplaceTempView('df_view')

# execute SQL query
df_result = spark.sql('select value from df_view where id=2')

# examine contents of result
df_result.show()

# get result as string
df_result.first().value

+------+
| value|
+------+
|value2|
+------+



'value2'

Although we’ll learn more about DataFrames next week, please try to find a way to count the rows in this DataFrame by looking at the API documentation. No worries, we’ll cover DataFrames in more detail next week.

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [17]:
df.count()

2

## Math for Statistics

In [18]:
rdd = sc.parallelize([101]+list(range(100))) #+[102,103]

### Mean

In [19]:
sum = rdd.sum()
n = rdd.count()
mean = sum/n
print (mean)

50.00990099009901


### Median

In [20]:
#rdd.sortBy(lambda x:x).collect
#rdd.sortBy(lambda x:x).zipWithIndex().collect()
sorterAndIndex = rdd.sortBy(lambda x:x).zipWithIndex().map(lambda value : (value[1],value[0]))
n = sorterAndIndex.count()
#print (sorterAndIndex.collect())
#print (n)
if (n % 2 == 1):
    index = (n-1)/2
    print (sorterAndIndex.lookup(index))
else:
    index1 = (n/2)-1
    index2 = n/2
    value1 = sorterAndIndex.lookup(index1)[0]
    value2 = sorterAndIndex.lookup(index2)[0]
    print ((value1+value2)/2)

[50]


### 2nd moment - Standard Deviation

In [21]:
rdd2 = sc.parallelize([49]*100+[100])

sum = rdd2.sum()
n = rdd2.count()
mean = sum/n
print (mean)

49.504950495049506


In [22]:
from math import sqrt
sd = sqrt(rdd2.map(lambda x:pow(x-mean,2)).sum()/n)
sd

5.049504950495049

### 3rd moment - Skewness

In [23]:
rdd3 = sc.parallelize(list(range(100))+[-1000]*1000)

##Mean
sum = rdd3.sum()
n = rdd3.count()
mean = sum/n
print ('mean:',mean)

sd = sqrt(rdd3.map(lambda x:pow(x-mean,2)).sum()/n)
print ('sd:',sd)

mean: -904.5909090909091
sd: 301.8355450920072


In [24]:
skewness = 1/n*rdd3.map(lambda x:pow(x-mean,3)/pow(sd,3)).sum()
skewness

2.8503857144434037

### 4th moment - Kurtosis

In [25]:
kurtosis = 1/n*rdd3.map(lambda x:pow(x-mean,4)/pow(sd,4)).sum()
kurtosis

9.134733566834043

### Covariance

In [27]:
rddX = sc.parallelize(range(100))
#rddY = sc.parallelize(range(100))
rddY = sc.parallelize(reversed(range(100)))

import random
#rddX = sc.parallelize(random.sample(range(100),100))
#rddY = sc.parallelize(random.sample(range(100),100))

#rddX = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
#rddY = sc.parallelize([7,6,5,4,5,6,7,8,9,10])

meanX = rddX.sum()/rddX.count()
meanY = rddY.sum()/rddY.count()

rddXY = rddY.zip(rddX)
rddXY.take(10)
covXY = rddXY.map(lambda x : (x[0]-meanX)*(x[1]-meanY)).sum()/rddXY.count()
covXY

-833.25

In [28]:
rddX.getNumPartitions()

2

In [29]:
rddY.getNumPartitions()

2

### Correlation

In [30]:
n = rddXY.count()
sdX = sqrt(rddX.map(lambda x:pow(x-meanX,2)).sum()/n)
sdY = sqrt(rddY.map(lambda x:pow(x-meanY,2)).sum()/n)
print(sdX)
print(sdY)

corrXY = covXY/(sdX*sdY)
corrXY

28.86607004772212
28.86607004772212


-1.0

### Correlation Matrix

In [31]:
from math import sqrt
from pyspark.mllib.stat import Statistics

column1 = sc.parallelize(range(100))
column2 = sc.parallelize(range(100,200))
column3 = sc.parallelize(reversed(range(100)))
column4 = sc.parallelize(random.sample(range(100),100))

data = column1.zip(column2).zip(column3).zip(column4).map(lambda dat: (dat[0][0][0],dat[0][0][1],dat[0][1],dat[1]))
print(Statistics.corr(data))

[[ 1.          1.         -1.         -0.04724872]
 [ 1.          1.         -1.         -0.04724872]
 [-1.         -1.          1.          0.04724872]
 [-0.04724872 -0.04724872  0.04724872  1.        ]]


## Statistics and transfomrations using DataFrames

In [32]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-07-10 15:59:43--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-07-10 15:59:43--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-07-10 15:59:43--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Savin

In [33]:
df.show()
df.printSchema()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [34]:
spark.sql('select class,count(*) from df group by class').show()

+--------------+--------+
|         class|count(1)|
+--------------+--------+
| Use_telephone|   15225|
| Standup_chair|   25417|
|      Eat_meat|   31236|
|     Getup_bed|   45801|
|   Drink_glass|   42792|
|    Pour_water|   41673|
|     Comb_hair|   23504|
|          Walk|   92254|
|  Climb_stairs|   40258|
| Sitdown_chair|   25036|
|   Liedown_bed|   11446|
|Descend_stairs|   15375|
|   Brush_teeth|   29829|
|      Eat_soup|    6683|
+--------------+--------+



In [35]:
df.groupBy('class').count().show()

+--------------+-----+
|         class|count|
+--------------+-----+
| Use_telephone|15225|
| Standup_chair|25417|
|      Eat_meat|31236|
|     Getup_bed|45801|
|   Drink_glass|42792|
|    Pour_water|41673|
|     Comb_hair|23504|
|          Walk|92254|
|  Climb_stairs|40258|
| Sitdown_chair|25036|
|   Liedown_bed|11446|
|Descend_stairs|15375|
|   Brush_teeth|29829|
|      Eat_soup| 6683|
+--------------+-----+



Let’s create a bar plot from this data. We’re using the pixidust library, which is Open Source, because of its simplicity. But any other library like matplotlib is fine as well. 

In [36]:
#!pip install pixiedust
import pixiedust
from pyspark.sql.functions import col
counts = df.groupBy('class').count().orderBy('count')
display(counts)

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


DataFrame[class: string, count: bigint]

This looks nice, but it would be nice if we can aggregate further to obtain some quantitative metrics on the imbalance like, min, max, mean and standard deviation. If we divide max by min we get a measure called minmax ration which tells us something about the relationship between the smallest and largest class. Again, let’s first use SQL for those of you familiar with SQL. Don’t be scared, we’re used nested sub-selects, basically selecting from a result of a SQL query like it was a table. All within on SQL statement.

In [37]:
spark.sql('''
    select 
        *,
        max/min as minmaxratio -- compute minmaxratio based on previously computed values
        from (
            select 
                min(ct) as min, -- compute minimum value of all classes
                max(ct) as max, -- compute maximum value of all classes
                mean(ct) as mean, -- compute mean between all classes
                stddev(ct) as stddev -- compute standard deviation between all classes
                from (
                    select
                        count(*) as ct -- count the number of rows per class and rename it to ct
                        from df -- access the temporary query table called df backed by DataFrame df
                        group by class -- aggrecate over class
                )
        )   
''').show()

+----+-----+------------------+------------------+-----------------+
| min|  max|              mean|            stddev|      minmaxratio|
+----+-----+------------------+------------------+-----------------+
|6683|92254|31894.928571428572|21284.893716741157|13.80427951518779|
+----+-----+------------------+------------------+-----------------+



The same query can be expressed using the DataFrame API. Again, don’t be scared. It’s just a sequential expression of transformation steps. You now an choose which syntax you like better.

In [38]:
from pyspark.sql.functions import col, min, max, mean, stddev

df \
    .groupBy('class') \
    .count() \
    .select([ 
        min(col("count")).alias('min'), 
        max(col("count")).alias('max'), 
        mean(col("count")).alias('mean'), 
        stddev(col("count")).alias('stddev') 
    ]) \
    .select([
        col('*'),
        (col("max") / col("min")).alias('minmaxratio')
    ]) \
    .show()

+----+-----+------------------+------------------+-----------------+
| min|  max|              mean|            stddev|      minmaxratio|
+----+-----+------------------+------------------+-----------------+
|6683|92254|31894.928571428572|21284.893716741157|13.80427951518779|
+----+-----+------------------+------------------+-----------------+



Now it’s time for you to work on the data set. First, please create a table of all classes with the respective counts, but this time, please order the table by the count number, ascending.

In [39]:
spark.sql('select class,count(*) as count from df group by class order by count').show()

+--------------+-----+
|         class|count|
+--------------+-----+
|      Eat_soup| 6683|
|   Liedown_bed|11446|
| Use_telephone|15225|
|Descend_stairs|15375|
|     Comb_hair|23504|
| Sitdown_chair|25036|
| Standup_chair|25417|
|   Brush_teeth|29829|
|      Eat_meat|31236|
|  Climb_stairs|40258|
|    Pour_water|41673|
|   Drink_glass|42792|
|     Getup_bed|45801|
|          Walk|92254|
+--------------+-----+



Pixiedust is a very sophisticated library. It takes care of sorting as well. Please modify the bar chart so that it gets sorted by the number of elements per class, ascending. Hint: It’s an option available in the UI once rendered using the display() function.

In [40]:
from pyspark.sql.functions import min

# create a lot of distinct classes from the dataset
classes = [row[0] for row in df.select('class').distinct().collect()]

# compute the number of elements of the smallest class in order to limit the number of samples per calss
min = df.groupBy('class').count().select(min('count')).first()[0]

# define the result dataframe variable
df_balanced = None

# iterate over distinct classes
for cls in classes:
    
    # only select examples for the specific class within this iteration
    # shuffle the order of the elements (by setting fraction to 1.0 sample works like shuffle)
    # return only the first n samples
    df_temp = df \
        .filter("class = '"+cls+"'") \
        .sample(False, 1.0) \
        .limit(min)
    
    # on first iteration, assing df_temp to empty df_balanced
    if df_balanced == None:    
        df_balanced = df_temp
    # afterwards, append vertically
    else:
        df_balanced=df_balanced.union(df_temp)

Please verify, by using the code cell below, if df_balanced has the same number of elements per class. You should get 6683 elements per class.

In [41]:
df_balanced.createOrReplaceTempView('df_balanced')
spark.sql('select class,count(*) as count from df_balanced group by class').show()

+--------------+-----+
|         class|count|
+--------------+-----+
| Use_telephone| 6683|
| Standup_chair| 6683|
|      Eat_meat| 6683|
|     Getup_bed| 6683|
|   Drink_glass| 6683|
|    Pour_water| 6683|
|     Comb_hair| 6683|
|          Walk| 6683|
|  Climb_stairs| 6683|
| Sitdown_chair| 6683|
|   Liedown_bed| 6683|
|Descend_stairs| 6683|
|   Brush_teeth| 6683|
|      Eat_soup| 6683|
+--------------+-----+



## Parallelism in Apache Spark - Exam

In [42]:
## 2
df1 = spark.createDataFrame([1,2,4,5,34,1,32,4,34,2,1,3], "int").toDF("data")
df1.printSchema()
#df1.show()

df1.createOrReplaceTempView('df1')
mean = spark.sql('select mean(data) as mean from df1').collect()[0]['mean']
mean

root
 |-- data: integer (nullable = true)



10.25

In [43]:
## 3.
rdd = sc.parallelize([1,2,4,5,34,1,32,4,34,2,1,3])

sorterAndIndex = rdd.sortBy(lambda x:x).zipWithIndex().map(lambda value : (value[1],value[0]))
n = sorterAndIndex.count()
#print (sorterAndIndex.collect())
#print (n)
if (n % 2 == 1):
    index = (n-1)/2
    print (sorterAndIndex.lookup(index))
else:
    index1 = (n/2)-1
    index2 = n/2
    value1 = sorterAndIndex.lookup(index1)[0]
    value2 = sorterAndIndex.lookup(index2)[0]
    print ((value1+value2)/2)

    
#df1.approxQuantile("data", [0.5], 0.0001)

3.5


In [44]:
## 5
df1 = spark.createDataFrame([34,1,23,4,3,3,12,4,3,1], "int").toDF("data")

std = spark.sql('select std(data) as std from df1').collect()[0]['std']
std

13.987819376482204

In [45]:
## 7
spark.sql("""
SELECT 
    ( 1/count(data) ) * SUM ( POWER(data-%s,4)/POWER(%s,4) ) as kurtosis from df1
                    """ %(mean,std)).first().kurtosis

1.9546000280182132

In [46]:
## 8
spark.sql("""
SELECT 
    ( 1/count(data) ) * SUM ( POWER(data-%s,3)/POWER(%s,3) ) as skewness from df1
                    """ %(mean,std)).first().skewness

0.9917330791185353

In [47]:
## 9
#dfa = spark.createDataFrame( (1,2,3,4,5,6,7,8,9,10),(7,6,5,4,5,6,7,8,9,10), ).toDF(["dataA","datab"])

rddX = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rddY = sc.parallelize([7,6,5,4,5,6,7,8,9,10])

meanX = rddX.sum()/rddX.count()
meanY = rddY.sum()/rddY.count()

rddXY = rddY.zip(rddX)
rddXY.take(10)
covXY = rddXY.map(lambda x : (x[0]-meanX)*(x[1]-meanY)).sum()/rddXY.count()
print('covariance:',covXY)


n = rddXY.count()
sdX = sqrt(rddX.map(lambda x:pow(x-meanX,2)).sum()/n)
sdY = sqrt(rddY.map(lambda x:pow(x-meanY,2)).sum()/n)
#print(sdX)
#print(sdY)

corrXY = covXY/(sdX*sdY)
print('correlation:',corrXY)

covariance: 2.21
correlation: 0.42945017416576226
